# Converting .vtu to .obj

In [1]:
import os
import pyvista as pv

import trimesh
from scripts.dataset import CustomDataset
from scripts.models import Segmentation_Network_full

import torch
import torch.nn as nn

model = torch.load('./models/best.pt').cpu()
model.eval()


dir_in = './data_for_inference/vtu/'
dir_out = './data_for_inference/obj/'

In [2]:
for i in os.listdir(dir_in):
    mesh = pv.read(dir_in + i)    
    surface = mesh.extract_surface()
    name = i[:-4]
    pv.save_meshio(filename=dir_out + name + '.obj', mesh = surface, file_format='obj')

# Infering on .obj

In [3]:
# Choose .obj to infer on
os.listdir(dir_out)

['1.obj', '2.obj']

In [4]:
name = '2.obj'

In [5]:
datagen = CustomDataset(name = name,
                        data_dir = dir_out,
                  device='cpu',
                  count=20,
                  rotate_angle=1,
                  dataset = 'custom',
                  num_classes=3,
                  inference = True,
                  return_points = True)

In [6]:
sdf, colors, points = datagen.get_data()
predicted = model(sdf)

In [7]:
mesh = trimesh.load(dir_out+name)

In [8]:
def save_mesh_with_color(mesh, new_colors, path):
    orig_colors = mesh.visual.vertex_colors
    new_colors = torch.argmax(new_colors, dim=1)
    for i in range(len(new_colors)):
        if new_colors[i] == 0:
            orig_colors[i] = [224, 0, 0, 255]
        elif new_colors[i] == 1:
            orig_colors[i] = [0, 224, 0, 255]
        else:
    	    orig_colors[i] = [0, 0, 224, 255]
        
    mesh.visual.vertex_colors = orig_colors
    file = trimesh.exchange.export.export_mesh(mesh, 'obj', None)
    with open(path, mode='w') as f:
        f.write(file)

In [9]:
save_mesh_with_color(mesh, predicted, path=f'./predictions/{name[:-4]}_predicted.obj')